In [51]:
import pandas as pd
from sklearn.linear_model import LinearRegression

In [52]:
data = pd.read_pickle("./cleaned_data.pkl")

In [53]:
data['location'].value_counts()

sliema             7326
gozo               6308
mosta              4774
st julians         4152
mellieħa           3546
qawra              2985
attard             2974
birkirkara         2802
msida              2558
naxxar             2476
gżira              2427
st paul's bay      2172
żabbar             1898
ibraġ              1809
st venera          1659
qormi              1197
buġibba            1066
valletta           1048
pietà              1043
lija               1041
xemxija             967
baħar iċ-ċagħaq     830
birżebbuġa          772
swatar              658
birgu               633
bormla              517
isla                490
baħrija             250
hamrun               15
żebbug               14
għarghur              3
st lucia              1
Name: location, dtype: int64

In [40]:
dummies = pd.get_dummies(data=data, columns=["type"])

In [41]:
dummies

,id,price,location,type_0,type_1,type_2,type_3,type_4,type_5,type_6
2,1101106,105000.0,8,1,0,0,0,0,0,0
4,1101108,260000.0,19,0,0,1,0,0,0,0
9,1100419,149000.0,0,1,0,0,0,0,0,0
10,1100420,680000.0,0,0,0,0,0,0,0,1
11,1100421,278000.0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
714,1169520,260000.0,22,0,0,0,0,0,1,0
716,1169522,580000.0,23,0,0,0,0,0,0,1
718,1169524,170000.0,23,1,0,0,0,0,0,0
722,1169481,150000.0,16,1,0,0,0,0,0,0


In [44]:
model = LinearRegression()
model.fit(df[["location", "type"]], data)

In [45]:
X = dummies.drop("price", axis="columns") # drop the dependent variable

In [46]:
y = dummies.price # dependent variable which my regressor is trying to predict

In [47]:
model.fit(X,y)

LinearRegression()

In [49]:
model.intercept_

-138947239.8685635

In [30]:
from sklearn.preprocessing import LabelEncoder

In [31]:
le = LabelEncoder()

In [32]:
datalabelencoder = data

In [39]:
datalabelencoder.location = le.fit_transform(datalabelencoder.location)
datalabelencoder.type = le.fit_transform(datalabelencoder.type)
datalabelencoder

,id,price,location,type
2,1101106,105000.0,8,0
4,1101108,260000.0,19,2
9,1100419,149000.0,0,0
10,1100420,680000.0,0,6
11,1100421,278000.0,0,0
...,...,...,...,...
714,1169520,260000.0,22,5
716,1169522,580000.0,23,6
718,1169524,170000.0,23,0
722,1169481,150000.0,16,0
